### Microsoft Studio Recommendations Analysis

Author: Mark Njagi

#### Overview
In this project, I undertake intensive exploratory data analysis to help Microsoft identify niche areas to exploit within the movie industry. The dataset used is sourced from The Movie Database API. Analysis of this data reveals that over the past few years, the income raked by movie studios has grown exponentially. This finding backs up Microsoft's decision to tap into this market space to diversify its income stream, and the recommendations provided in this analysis will assist the management at Microsoft understand what steps to take to best tap into this space.

#### Business Problem
Microsoft is a leading software company that is best known for its wide product range eg. the Windows operating system, the Microsoft Office Suite, Internet Explorer and the Xbox console and gaming line. Leveraging on its global outreach, Microsoft has been able to maintain a competitive edge over similar service providers such as Sony and Google. In the movie space however, Microsoft has severely struggled to assert its dominance. The Xbox Entertainment Studios, which once harboured Microsoft's grand ambitions to conquer the movie space in 2012, was shut down two years later following reports of disorganization and struggle to clinch important business deals (more information here - https://thenextweb.com/news/microsoft-closing-xbox-entertainment-studios). 


This analysis aims to help Microsoft management make a rebound into the industry, learn from its previous mistakes and incorporate fresh, renewed ideas for its growth.

#### Data Understanding
This analysis uses data compiled from The Movie Dataset API(attach link to database), which is a popular, community-based database which allows users to search for movie and TV shows information by title or unique user ids.

To view the data, we first have to import the necessary libraries. 

In [1]:
# To interact with the TMDb API
import json
import requests
from tmdbv3api import Movie

# To read csv, json files
import pandas as pd

# To work with path files
import os
import sys
sys.path.append('code/datacleaning')
sys.path.append('code')

# For plotting
import matplotlib.pyplot as plt
import seaborn as sns

# TMDB Credentials
from credentials import username, api_key

# To render matplotlib figures inside the notebook
%matplotlib inline

Reading the csv file and saving the result to a variable; movie_data

In [2]:
# Reading the csv file and saving the result to a variable df
df = pd.read_csv('zippedData/zippedData/tmdb.movies.csv.gz')

# To display the first five rows of our dataset
df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [3]:
print (f'The dataset contains {df.shape[0]} rows and {df.shape[1]} columns')

The dataset contains 26517 rows and 10 columns


An individual explanation of each of the columns of the uncleaned dataset is as follows:

1. __genre_ids__ - A collection of id numbers each of which is assigned to a specific genre name. for example, id 12 is mapped to the 'Adventure' genre category.
2. __id__ - a unique identifier of each movie within the dataset and API database.
3. __original_language__ - the primary language spoken in the movie's country of production.
4. __original title__ - the original name of the movie in the language of the country of origination. The title column shows the translation of the original title to English. For example, the movie Contes de Juillet whose country of origin is France shows July Tales in the title column.
5. __popularity__ - The lifetime rating of a movie, calculated based on several factors such as number of views, number of votes for the movie, number of users who marked it as a 'favorite', as well as number of users who added the movie to their 'Watchlist'. The popularity score of a movie is calculated continuously from the date of release.
6. __release_date__ - The date on which the movie premiered.
7. __vote_average__ - The average vote of a movie, on a scale of 1 to 10, calculated as the sum of total votes received divided by the number of people who participated in the voting exercise (__vote_count__).


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB





4. Converting the Release_Date to a datetime object in order to extract the year, month, and day of release.
5. Feature Engineering - I added two new columns('budget') and ('revenue') which showed the production budget of the movie and the revenue generated. Further, I created another column ('Profit') which is calculated as the difference between the revenue generated and the production budget of the movie. 

The code is as shown below:

#### Data Cleaning
As illustrated above, it is evident that our data will have to undergo cleaning in order to transform it to a usable function. The data cleaning process contained several steps, the code for which is included here(link to data cleaning function):

1. Dropping Unnecessary Columns - I dropped the Unnamed column since it is repetitive and contains info from the index column. Furthermore, I dropped the original_title column and left the title column for use in my data analysis.

In [5]:
import drop_unnecessary_columns as duc

In [6]:
duc.drop_unnecessary_columns(df, columns = ['Unnamed: 0', 'original_title'])

Column Unnamed: 0 dropped.
Column original_title dropped.


2. Converting the Genre_ID columns to actual Genre Names. To achieve this, I created a dictionary using data from the TMDb API which mapped each genre_id to the corresponding genre name and mapped this on each element in the genre_id column.

3. Removing Duplicated Rows - I checked for values where the movie_id appeared more than once in the rows and dropped extra instances.

In [7]:
import remove_duplicates as rd

In [8]:
rd.remove_duplicates(df)

This dataset has 1020 duplicated values
Below is a sample of duplicates: 


NameError: name 'df' is not defined

In [ ]:
df

The following code generates a mapping dictionary that will be used to map each genre_id to its genre_name

In [ ]:
# Retrieving the genres data from the TMDb API
response = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}')
parse_json = json.loads(response.text)
results = parse_json['genres']

# Creating a dictionary with dict comprehension
genres_map = {result['id']: result['name'] for result in results}

#Viewing genres_map
genres_map

#### Data Preparation

#### Analysis

#### Conclusions

#### Recommendations